In [29]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
from WhatIfAnalysis import GoalSeek
# import sectionproperties as shape

from ASCE import load_combos

from wood_design.sawn_lumber.ref_design_values.table_4a import table_4a_load
from wood_design.sawn_lumber.ref_design_values.C_F import C_F_load
from wood_design.connections.NDS_design_factors import group_factor
table_4a = table_4a_load(species="DF")

In [30]:
enveloped_pressure = load_combos.ASD(D=15, L=20, W=36)
canopy_width = 5 #ft
canopy_length = 10 #ft
vertical_reaction_trib = enveloped_pressure * canopy_width/2 * canopy_length/2 #lbs
f"vertical reaction at corner = {vertical_reaction_trib} lbs"

'vertical reaction at corner = 577.5 lbs'

In [31]:
θ = arctan(7.375/12) #rad
f"angle of brace = {round(rad2deg(θ), 2)} deg"

'angle of brace = 31.57 deg'

In [32]:
brace_tension = vertical_reaction_trib / sin(θ)
f"tension in brace = {round(brace_tension, 2)}"

'tension in brace = 1102.94'

<div style="page-break-after: always;"></div>

# Canopy Connection Design

## Screwed Attachment Capacity

#### Wind Combo Demand

In [33]:
T_x = 1500 #lbs
T_y = 800 #lbs

α = arctan(12/7.375) #rad
f"{T_x} lbs, {round(T_y, 2)} lbs"

'1500 lbs, 800 lbs'

In [34]:
brace_tension = sqrt(T_x**2 + T_y**2)
f"tension in brace = {round(brace_tension, 2)}"

'tension in brace = 1700.0'

#### Wind Combo Capacity

In [35]:
Z_nominal = 180 #lbs (Table 12K)
W_nominal = 305 #lbs/in
shaft_length = 3 #in

In [22]:

def dowel_shear(Z_nominal, C_D=1.0, C_M=1.0, C_t=1.0, C_g=1.0, C_delta=1.0, C_eg=1.0, C_di=1.0, toe_nailing="N"):
    '''
    - NDS Table 11.3.1
    - All adjustment factors are 1.0 by default
    '''
    if toe_nailing == "Y":
        C_tn = 0.83   
    else:
        C_tn = 1.0 
    Z_factored = Z_nominal * C_D * C_M * C_t * C_g * C_delta * C_eg * C_di * C_tn
    return Z_factored


def dowel_withdrawal(W_nominal, penetration=1.0, C_D=1.0, C_M=1.0, C_t=1.0, C_eg=1.0, toe_nailing="N"):
    W = W_nominal * penetration
    if toe_nailing == "Y":
        C_tn = 0.67
    else:
        C_tn = 1.0
    W_factored = W_nominal * C_D * C_M**2 * C_t * C_eg * C_tn
    return W_factored


# alpha angle from pure shear in degrees
def combined_scew_Z(Z_pr, W_pr, penetration, alpha, conv="deg"):
    if conv == "deg":
        alpha_rad = np.deg2rad(alpha)
    elif conv == "rad":
        alpha_rad = alpha
    Wp = W_pr * penetration
    Z_alpha_pr = Wp * Z_pr / (Wp * np.cos(alpha_rad)**2 + Z_pr * np.sin(alpha_rad)**2)
    return Z_alpha_pr

In [23]:
Z_pr = dowel_shear(Z_nominal, C_D=1.6)
W_pr = dowel_withdrawal(W_nominal, C_D=1.6)
f"Z' = {round(Z_nominal, 2)} lbs, W'*p = {round(W_pr * shaft_length, 2)} lbs"

"Z' = 180 lbs, W'*p = 1464.0 lbs"

In [24]:
Zα_pr = W_pr * shaft_length * Z_pr \
	/ (W_pr * shaft_length * cos(α)**2 + Z_pr * sin(α)**2)
f"Zα' = {round(Zα_pr, 2)} lbs"

"Zα' = 690.73 lbs"

In [25]:
total_conn_capacity = Zα_pr * 4
f"total lag screw capacity = {round(total_conn_capacity, 2)} lbs"

'total lag screw capacity = 2762.94 lbs'

In [26]:
f"DCR = {round(brace_tension / total_conn_capacity, 3)}"

'DCR = 0.615'

## Wood Blocking Attachment

--> Use Simpson A34 Connector:

In [27]:
Va_conn = 550 * 2 #lbs

In [28]:
P = T_x #lbs
l = 16/12 #ft
b = 10/12 #ft
R1 = P*b/l #lbs
f"max R1 allowed = {R1} lbs"

'max R1 allowed = 937.5 lbs'